In [1]:
import sys
from traceback import format_exc
sys.path.append("../")
from trace_decorator import trace

In [2]:
s = "abcdefg"
ix = s.find("bc")
s[ix:]

'bcdefg'

In [3]:
global_var = "Simon Says"

@trace
def a_function():

    i = 0
    i += 1

    s = "a"
    s += "b"
    s += "c"

    l = []
    l.append(i)
    l.append(s)
    l.append(l)

    t = global_var
#     t = 999

    print(s + s + s)
    for i in range(4):
        i_sq_a_a_a_a = i ** 2
    
    return t

a_function()

*** Trace Output *** -----------------------------------------------------------
    @trace
    def a_function():                                  | 
                                                       | 
        i = 0                                          | i = 0 
        i += 1                                         | i = 1 
                                                       | 
        s = "a"                                        | s = 'a' 
        s += "b"                                       | s = 'ab' 
        s += "c"                                       | s = 'abc' 
                                                       | 
        l = []                                         | l = [] 
        l.append(i)                                    | l = [1] 
        l.append(s)                                    | l = [1, 'abc'] 
        l.append(l)                                    | l = [1, 'abc', [...]] l = [1, 'abc', [...]] 
                                         

'Simon Says'

In [4]:
len("*** Trace Output *** " + "-" * 70), len("*** END Trace Output ***" + "-" * 67)

(91, 91)

In [5]:
from code_tracer import CodeTracer

from IPython import get_ipython
from IPython.core.magic import Magics, magics_class, cell_magic

@magics_class
class TraceCellMagic(Magics):

    # override default constructor so we can pass state between magics
    def __init__(self, shell):
        # You must call the parent constructor
        super(TraceCellMagic, self).__init__(shell)

    @cell_magic
    def trace(self, line, cell):

        source = cell
        ipython = get_ipython()

        dump = True
        max_width = 2000

        tracer = CodeTracer()
        tracer.max_width = max_width

        trace = tracer.trace_code(source, global_context=ipython.user_ns, local_context=dict(), dump=dump)

        print("*** Trace Output *** ".ljust(80,"-"))
        print()
        print(trace)
        print()
        print("*** END Trace Output *** ".ljust(80,"-"))
        
#         return trace

def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    # You can register the class itself without instantiating it.  IPython will
    # call the default constructor on it.
    ipython.register_magics(TraceCellMagic)

In [6]:
ipy = get_ipython()
load_ipython_extension(ipy)

In [7]:
gloabl_var = "I am a global var"

In [11]:
%%trace
import pandas as pd

rows = [{"ix": i, "i**2": i**2, "2i": 2*i} for i in range(10)]

df = pd.DataFrame(rows)

def add_2_things(a,b):
    return a + b

a = "ssss"
b = 121212

nums = add_2_things(1,3)
strings = add_2_things("aaa","bbb")

t = global_var
print(t)
print(t)
print(t)

d = "asasasasas"

*** Trace Output *** -----------------------------------------------------------

    import pandas as pd                                            | 
                                                                   | 
    rows = [{"ix": i, "i**2": i**2, "2i": 2*i} for i in range(10)] | rows = [{'ix': 0, 'i**2': 0, '2i': 0}, {'ix': 1, 'i**2': 1, '2i': 2}, {'ix': 2, 'i**2': 4, '2i': 4}, {'ix': 3, 'i**2': 9, '2i': 6}, {'ix': 4, 'i**2': 16, '2i': 8}, {'ix': 5, 'i**2': 25, '2i': 10}, {'ix': 6, 'i**2': 36, '2i': 12}, {'ix': 7, 'i**2': 49, '2i': 14}, {'ix': 8, 'i**2': 64, '2i': 16}, {'ix': 9, 'i**2': 81, '2i': 18}] 
                                                                   | 
    df = pd.DataFrame(rows)                                        | df = ix i**2 2i 0 0 0 0 1 1 1 2 2 2 4 4 3 3 9 6 4 4 16 8 5 5 25 10 6 6 36 12 7 7 49 14 8 8 64 16 9 9 81 18 
                                                                   | 
    def add_2_things(a,b):                                    

## TODO
- If max line width exceeded, just truncate with ....
- Add a function and method that returns the dataframe for querying
- Add option to add call counts or time to the trace output
- Add a profiling option to profile the code and produce a profile report instead (number of calls or time spent)